## Capturing : Capture the geometry from the sampling and convert to file array ##
Maintain: Demo and test only. <br/>
For real calculation, a separate UNIX shell script that run the calculation and another separate Python script to capture the results (either directly from the .xyz file or summary from csv file) is required


In [1]:
test_name = "glycine_non_opt_350_20000s"

In [2]:
import os
import sys
import pandas as pd

sys.path.insert(0, "..")

In [3]:
from ase.calculators.nwchem import NWChem
os.environ['ASE_NWCHEM_COMMAND'] = 'mpirun -n 8 nwchem PREFIX.nw > PREFIX.out'
nwchem_calc = NWChem(label='calc/nwchem',
              maxiter=2000,
              xc='b3lyp',
              basis='6-31G**')



In [4]:
import numpy as np
from ase.units import Hartree, mol, kcal, Bohr
def nwchem_compute(md_samples_arr, nwchem_calc):
    """
    Take the array with the atoms, compute the DFT energy for all the given configurations.
    """
    
    md_dft_energy_arr = np.zeros(md_samples_arr.size + 1)
    md_dft_force_arr = pd.Series([]) # Use pd.Series since molecule size may vary
    for index, atoms in md_samples_arr.items():
        atoms.set_calculator(nwchem_calc)
        potent_e = atoms.get_potential_energy()
        force_arr = atoms.get_forces()
        print(index)
        print(potent_e)
        md_dft_energy_arr[index] = potent_e
        md_dft_force_arr[index] = force_arr
    
    return md_dft_energy_arr, md_dft_force_arr

def get_energy_force_mol(atoms, calc):
    """Get the energy for each molecule for a particular molecule (atoms)
    
            Args:
                atoms: an ase.Atoms object
                calc: an ase.calculator
                
            Outputs:
                poten_e*: potential energy for the molecule
                force_arr*: force for the molecule
                
                * Both wrapped in a Pandas.Series() object
    
    Comments:
    1. For FileIO Calculator, May improve the performance by 
    getting all the character at once.
    
    """
    atoms.set_calculator(calc)
    poten_e = atoms.get_potential_energy()
    force_arr = atoms.get_forces()
    
    return pd.Series([poten_e, force_arr])
    
# def dft_compute(md_samples_arr, dft_calc):
#     """Compute the DFT Energy for the given array using the given calculator
    
#             Args:
#                 md_samplesArr: a pd.Series() of all the configurations 
#                 dft_calc:  a ase.calculator that will be assigned to the atoms
                
#             Outputs:
#                 md_dft_energy_arr: an numpy array of all the dft energy
    
#     Comments: 
#     1. Attempt to generalize in the naming.
#     2. Attempt to use element-wise operations in pandas.Series(), 
#     which may not work for smaller sets.
#     3. Untested. 
#     """
    
#     md_dft_energy_arr = np.zeros(md_samples_arr.size)
#     md_samples_arr.apply(

In [5]:
## Global Var:
relative_energy_ref_dft = 0
relative_energy_ref_dftb = 0

def get_relative_energy(energy_arr):
    """
    Get the relative energy with reference to the lowest point in all the energy. 
    
        Args:
            energy_arr: an np.array of energy (either dftb or dft) of shape (No. samples, )
    
    """
    lowest_energy = np.amin(energy_arr)
    relative_energy_arr = energy_arr - lowest_energy
    
    return relative_energy_arr, lowest_energy


In [6]:
from dftb_traj_io import read_dftb_atom_from_traj
from dftb_traj_io import read_dftb_md_energy

## Get the Hybrid Rotating Sample Training Set 
Combine the MD trajectory with different initial geometries rotated for 60 degree. 

In [7]:
##########
# WARNING: TEMPORARY!
# Use it only because the original file has been lost. 
##########
sample_file_list = [ 'glycine_190HOCO_400K_10000s', 'glycine_360HOCO_400K_10000s']
md_samplesArr = pd.Series([])
for sample_file in sample_file_list:
    sample_file_name = '../md_sampling/samples/' + sample_file  + '/geom.out.xyz'
    md_samplesArr_temp, md_Mulliken = read_dftb_atom_from_traj(sample_file_name, writeTrajectory=True)
    md_samplesArr = md_samplesArr.append(md_samplesArr_temp, ignore_index=True)
    print(md_Mulliken.shape)
print(md_samplesArr.shape)

(10001, 10)
(10001, 10)
(20002,)


In [8]:
import os 
os.getcwd()


'/home/junmian/Applications/YeltsinNet/md_capture'

In [9]:
md_samplesArr

0        (Atom('C', [0.622, -0.1192, 0.0383], index=0),...
1        (Atom('C', [0.62255955, -0.1199231, 0.03914185...
2        (Atom('C', [0.62307313, -0.12065897, 0.0399874...
3        (Atom('C', [0.62354007, -0.12140603, 0.0408360...
4        (Atom('C', [0.6239599, -0.12216273, 0.0416871]...
5        (Atom('C', [0.62433229, -0.12292757, 0.0425398...
6        (Atom('C', [0.62465714, -0.12369907, 0.0433934...
7        (Atom('C', [0.62493447, -0.12447582, 0.0442472...
8        (Atom('C', [0.62516454, -0.12525644, 0.0451004...
9        (Atom('C', [0.62534773, -0.12603961, 0.0459522...
10       (Atom('C', [0.62548464, -0.12682406, 0.0468016...
11       (Atom('C', [0.625576, -0.12760858, 0.04764779]...
12       (Atom('C', [0.62562273, -0.12839202, 0.0484898...
13       (Atom('C', [0.62562591, -0.12917327, 0.0493268...
14       (Atom('C', [0.62558676, -0.12995128, 0.0501577...
15       (Atom('C', [0.62550666, -0.13072507, 0.0509816...
16       (Atom('C', [0.62538712, -0.1314937, 0.05179754.

In [10]:
sys.path.insert(0, "../Calculator")

## Training Notice:
Select 3000 configurations from the sampling and run the calculation. 

In [11]:
n_samples = len(md_samplesArr) - 1

n_train = 3000 

train_idx = np.linspace(0, n_samples, n_train, dtype=np.int32)
print(train_idx.shape)

(3000,)


In [12]:
print(train_idx)

[    0     6    13 ... 19987 19994 20001]


In [13]:
md_train_arr = md_samplesArr[train_idx]
print(md_train_arr.shape)
print(type(md_train_arr))

(3000,)
<class 'pandas.core.series.Series'>


## Calculate Energy 
Calculate the Energy using DFTB and Higher-Level Quantum Chemistry Potential. 

In [14]:
# def calc_compute(md_samples_arr, nwchem_calc, train_idx):
#     """
#     Take the array with the atoms, compute the DFT energy for all the given configurations.
    
#     (Seems pandas series has some problem with indexing )
#     """
    
#     md_dft_energy_arr = np.zeros(md_samples_arr.size + 1)
#     md_dft_force_arr = pd.Series([]) # Use pd.Series since molecule size may vary
#     for index, atoms in m:
#         atoms.set_calculator(nwchem_calc)
#         potent_e = atoms.get_potential_energy()
#         force_arr = atoms.get_forces()
#         print(index)
#         print(potent_e)
#         md_dft_energy_arr[index] = potent_e
#         md_dft_force_arr[index] = force_arr
    
#     return md_dft_energy_arr, md_dft_force_arr

def nwchem_compute(md_samples_arr, nwchem_calc):
    """
    Take the array with the atoms, compute the DFT energy for all the given configurations.
    """
    
    md_dft_energy_arr = np.zeros(md_samples_arr.size)
    md_dft_force_arr = pd.Series([]) # Use pd.Series since molecule size may vary
    count = 0
    for index, atoms in md_samples_arr.items():
        atoms.set_calculator(nwchem_calc)
        potent_e = atoms.get_potential_energy()
        force_arr = atoms.get_forces()
        print('Index:', index, ' Count:',  count, " Potential Energy: ", potent_e)
        md_dft_energy_arr[count] = potent_e
        md_dft_force_arr[count] = force_arr
        count += 1
    
    return md_dft_energy_arr, md_dft_force_arr


In [15]:
from ASEDFTBMod import DftbMod
os.environ['DFTB_PREFIX'] = '/home/junmian/Applications/dftbplus/mio-1-1/'
os.environ['DFTB_COMMAND'] = '/home/junmian/Applications/dftbplus/_install/bin/dftb+'


# Set path to the model


dftb_calc = DftbMod(
                        Hamiltonian_SCC ='YES',
                        Hamiltonian_SCCTolerance = 1.0E-008,
                        Hamiltonian_MaxAngularMomentum_='',
                        Hamiltonian_MaxAngularMomentum_O='"p"',
                        Hamiltonian_MaxAngularMomentum_H='"s"',
                        Hamiltonian_MaxAngularMomentum_C='"p"',
                        Hamiltonian_MaxAngularMomentum_N='"p"',
                         )


In [16]:
#md_energy_arr, md_kinetic_energy_arr = read_dftb_md_energy('../md_sampling/md.out')
md_energy_arr, dft_force_arr = nwchem_compute(md_train_arr, dftb_calc)
md_energy_arr.shape
print(md_energy_arr)

Index: 0  Count: 0  Potential Energy:  -390.06926171383833
Index: 6  Count: 1  Potential Energy:  -390.060945796297
Index: 13  Count: 2  Potential Energy:  -390.0197603428419
Index: 20  Count: 3  Potential Energy:  -389.96635259219187
Index: 26  Count: 4  Potential Energy:  -389.93699260274076
Index: 33  Count: 5  Potential Energy:  -389.9381228626042
Index: 40  Count: 6  Potential Energy:  -389.9381830110273
Index: 46  Count: 7  Potential Energy:  -389.92259139541073
Index: 53  Count: 8  Potential Energy:  -389.92951569803085
Index: 60  Count: 9  Potential Energy:  -389.96710863732494
Index: 66  Count: 10  Potential Energy:  -389.96965977440954
Index: 73  Count: 11  Potential Energy:  -389.92934290360984
Index: 80  Count: 12  Potential Energy:  -389.907714982098
Index: 86  Count: 13  Potential Energy:  -389.91973433536094
Index: 93  Count: 14  Potential Energy:  -389.9265276728101
Index: 100  Count: 15  Potential Energy:  -389.92383917797684
Index: 106  Count: 16  Potential Energy:  -

Index: 900  Count: 135  Potential Energy:  -389.9289486644749
Index: 907  Count: 136  Potential Energy:  -389.96451805258243
Index: 913  Count: 137  Potential Energy:  -389.99389661195687
Index: 920  Count: 138  Potential Energy:  -389.9848483506882
Index: 927  Count: 139  Potential Energy:  -389.943862292803
Index: 933  Count: 140  Potential Energy:  -389.9180247192764
Index: 940  Count: 141  Potential Energy:  -389.9307610282578
Index: 947  Count: 142  Potential Energy:  -389.9423595575012
Index: 953  Count: 143  Potential Energy:  -389.8996067064338
Index: 960  Count: 144  Potential Energy:  -389.8702053414308
Index: 967  Count: 145  Potential Energy:  -389.9195516681593
Index: 973  Count: 146  Potential Energy:  -389.92219949113525
Index: 980  Count: 147  Potential Energy:  -389.87225611881456
Index: 987  Count: 148  Potential Energy:  -389.8496645390605
Index: 993  Count: 149  Potential Energy:  -389.87318510329914
Index: 1000  Count: 150  Potential Energy:  -389.91172526436924
In

Index: 1767  Count: 265  Potential Energy:  -389.9653428701888
Index: 1774  Count: 266  Potential Energy:  -389.9530587419411
Index: 1780  Count: 267  Potential Energy:  -389.91567064243975
Index: 1787  Count: 268  Potential Energy:  -389.90250626978144
Index: 1794  Count: 269  Potential Energy:  -389.9270437029087
Index: 1800  Count: 270  Potential Energy:  -389.97419257511393
Index: 1807  Count: 271  Potential Energy:  -390.0314810590427
Index: 1814  Count: 272  Potential Energy:  -390.05392493478115
Index: 1820  Count: 273  Potential Energy:  -390.06319146316764
Index: 1827  Count: 274  Potential Energy:  -390.06016989111345
Index: 1834  Count: 275  Potential Energy:  -390.02088566034206
Index: 1840  Count: 276  Potential Energy:  -389.96058275311947
Index: 1847  Count: 277  Potential Energy:  -389.90200342158306
Index: 1854  Count: 278  Potential Energy:  -389.9113691103174
Index: 1860  Count: 279  Potential Energy:  -389.9466093535432
Index: 1867  Count: 280  Potential Energy:  -3

Index: 2641  Count: 396  Potential Energy:  -389.9951729158079
Index: 2647  Count: 397  Potential Energy:  -389.93936089093603
Index: 2654  Count: 398  Potential Energy:  -389.8769478587022
Index: 2661  Count: 399  Potential Energy:  -389.8827734772356
Index: 2667  Count: 400  Potential Energy:  -389.9375715720269
Index: 2674  Count: 401  Potential Energy:  -389.9611522143011
Index: 2681  Count: 402  Potential Energy:  -389.9058610413363
Index: 2687  Count: 403  Potential Energy:  -389.9031472056196
Index: 2694  Count: 404  Potential Energy:  -389.94561126499815
Index: 2701  Count: 405  Potential Energy:  -389.91839405309764
Index: 2707  Count: 406  Potential Energy:  -389.8694196362625
Index: 2714  Count: 407  Potential Energy:  -389.87046024452206
Index: 2721  Count: 408  Potential Energy:  -389.93962186634326
Index: 2727  Count: 409  Potential Energy:  -389.9961046469555
Index: 2734  Count: 410  Potential Energy:  -390.0027887441512
Index: 2741  Count: 411  Potential Energy:  -389.9

Index: 3508  Count: 526  Potential Energy:  -389.90369701072177
Index: 3514  Count: 527  Potential Energy:  -389.8712769329363
Index: 3521  Count: 528  Potential Energy:  -389.8682783995491
Index: 3528  Count: 529  Potential Energy:  -389.893345244355
Index: 3534  Count: 530  Potential Energy:  -389.9014000726556
Index: 3541  Count: 531  Potential Energy:  -389.8693441131776
Index: 3548  Count: 532  Potential Energy:  -389.8560113043828
Index: 3554  Count: 533  Potential Energy:  -389.8910571872198
Index: 3561  Count: 534  Potential Energy:  -389.9168359844982
Index: 3568  Count: 535  Potential Energy:  -389.90416543385834
Index: 3574  Count: 536  Potential Energy:  -389.91241500383086
Index: 3581  Count: 537  Potential Energy:  -389.97004583291465
Index: 3588  Count: 538  Potential Energy:  -390.0145622022876
Index: 3594  Count: 539  Potential Energy:  -389.98840716892187
Index: 3601  Count: 540  Potential Energy:  -389.91935407413814
Index: 3608  Count: 541  Potential Energy:  -389.8

...

Index: 16279  Count: 2441  Potential Energy:  -389.9336924098955
Index: 16286  Count: 2442  Potential Energy:  -389.9283374025798
Index: 16292  Count: 2443  Potential Energy:  -389.8592412321013
Index: 16299  Count: 2444  Potential Energy:  -389.85132245896204
Index: 16306  Count: 2445  Potential Energy:  -389.9293032192972
Index: 16312  Count: 2446  Potential Energy:  -389.92112206985144
Index: 16319  Count: 2447  Potential Energy:  -389.7686733981101
Index: 16326  Count: 2448  Potential Energy:  -389.7078041549635
Index: 16332  Count: 2449  Potential Energy:  -389.8055672995507
Index: 16339  Count: 2450  Potential Energy:  -389.8185686558377
Index: 16346  Count: 2451  Potential Energy:  -389.7474638504095
Index: 16352  Count: 2452  Potential Energy:  -389.7549297878287
Index: 16359  Count: 2453  Potential Energy:  -389.79252659810845
Index: 16366  Count: 2454  Potential Energy:  -389.84546676459524
Index: 16372  Count: 2455  Potential Energy:  -389.91264820549617
Index: 16379  Count:

Index: 17119  Count: 2567  Potential Energy:  -389.8045150423145
Index: 17126  Count: 2568  Potential Energy:  -389.74066065705614
Index: 17133  Count: 2569  Potential Energy:  -389.7565017252227
Index: 17139  Count: 2570  Potential Energy:  -389.80901634296606
Index: 17146  Count: 2571  Potential Energy:  -389.8429410535277
Index: 17153  Count: 2572  Potential Energy:  -389.8275697516735
Index: 17159  Count: 2573  Potential Energy:  -389.83975211887787
Index: 17166  Count: 2574  Potential Energy:  -389.92470022807436
Index: 17173  Count: 2575  Potential Energy:  -389.9131062080379
Index: 17179  Count: 2576  Potential Energy:  -389.8142630214002
Index: 17186  Count: 2577  Potential Energy:  -389.80046495731494
Index: 17193  Count: 2578  Potential Energy:  -389.82831174168484
Index: 17199  Count: 2579  Potential Energy:  -389.76169488747564
Index: 17206  Count: 2580  Potential Energy:  -389.70006828113094
Index: 17213  Count: 2581  Potential Energy:  -389.7687310709693
Index: 17219  Cou

Index: 17960  Count: 2693  Potential Energy:  -389.83666331375014
Index: 17966  Count: 2694  Potential Energy:  -389.84800626511975
Index: 17973  Count: 2695  Potential Energy:  -389.7874787830902
Index: 17980  Count: 2696  Potential Energy:  -389.82115145928293
Index: 17986  Count: 2697  Potential Energy:  -389.8294316963977
Index: 17993  Count: 2698  Potential Energy:  -389.7964191172399
Index: 18000  Count: 2699  Potential Energy:  -389.79629695920005
Index: 18006  Count: 2700  Potential Energy:  -389.8212179751595
Index: 18013  Count: 2701  Potential Energy:  -389.82830873808115
Index: 18020  Count: 2702  Potential Energy:  -389.8150436837136
Index: 18026  Count: 2703  Potential Energy:  -389.88737258992984
Index: 18033  Count: 2704  Potential Energy:  -389.9319363770559
Index: 18040  Count: 2705  Potential Energy:  -389.8153181210371
Index: 18046  Count: 2706  Potential Energy:  -389.7692869286974
Index: 18053  Count: 2707  Potential Energy:  -389.8494726218951
Index: 18060  Count

Index: 18807  Count: 2820  Potential Energy:  -389.8937207838771
Index: 18813  Count: 2821  Potential Energy:  -389.79917553064774
Index: 18820  Count: 2822  Potential Energy:  -389.81764505415157
Index: 18827  Count: 2823  Potential Energy:  -389.9045569823395
Index: 18833  Count: 2824  Potential Energy:  -389.86243978590267
Index: 18840  Count: 2825  Potential Energy:  -389.7192698605745
Index: 18847  Count: 2826  Potential Energy:  -389.7084837438571
Index: 18853  Count: 2827  Potential Energy:  -389.75266204898384
Index: 18860  Count: 2828  Potential Energy:  -389.73767581406275
Index: 18867  Count: 2829  Potential Energy:  -389.75652437053
Index: 18873  Count: 2830  Potential Energy:  -389.8329076224066
Index: 18880  Count: 2831  Potential Energy:  -389.87787113051553
Index: 18887  Count: 2832  Potential Energy:  -389.8947520095419
Index: 18893  Count: 2833  Potential Energy:  -389.9513533580162
Index: 18900  Count: 2834  Potential Energy:  -389.90673309088015
Index: 18907  Count:

Index: 19647  Count: 2946  Potential Energy:  -389.7997366589574
Index: 19654  Count: 2947  Potential Energy:  -389.7298735093347
Index: 19660  Count: 2948  Potential Energy:  -389.7573538986014
Index: 19667  Count: 2949  Potential Energy:  -389.84797290252374
Index: 19674  Count: 2950  Potential Energy:  -389.86574484572196
Index: 19680  Count: 2951  Potential Energy:  -389.8574096169431
Index: 19687  Count: 2952  Potential Energy:  -389.87837684113947
Index: 19694  Count: 2953  Potential Energy:  -389.83665857219063
Index: 19700  Count: 2954  Potential Energy:  -389.7066314806338
Index: 19707  Count: 2955  Potential Energy:  -389.6759791184068
Index: 19714  Count: 2956  Potential Energy:  -389.7586379594608
Index: 19720  Count: 2957  Potential Energy:  -389.75265089729527
Index: 19727  Count: 2958  Potential Energy:  -389.7406774341382
Index: 19734  Count: 2959  Potential Energy:  -389.8582741933804
Index: 19740  Count: 2960  Potential Energy:  -389.950683363226
Index: 19747  Count: 

In [17]:
md_dft_energy_arr, md_dft_force_arr = nwchem_compute(md_train_arr, nwchem_calc)

Index: 0  Count: 0  Potential Energy:  -7739.78985743711
Index: 6  Count: 1  Potential Energy:  -7739.752347586365
Index: 13  Count: 2  Potential Energy:  -7739.645570660977
Index: 20  Count: 3  Potential Energy:  -7739.523221207866
Index: 26  Count: 4  Potential Energy:  -7739.455582484794
Index: 33  Count: 5  Potential Energy:  -7739.443434820621
Index: 40  Count: 6  Potential Energy:  -7739.450937374856
Index: 46  Count: 7  Potential Energy:  -7739.439523096531
Index: 53  Count: 8  Potential Energy:  -7739.427336106354
Index: 60  Count: 9  Potential Energy:  -7739.415490179201
Index: 66  Count: 10  Potential Energy:  -7739.369080901039
Index: 73  Count: 11  Potential Energy:  -7739.295232845123
Index: 80  Count: 12  Potential Energy:  -7739.285012088992
Index: 86  Count: 13  Potential Energy:  -7739.340931114938
Index: 93  Count: 14  Potential Energy:  -7739.418020644899
Index: 100  Count: 15  Potential Energy:  -7739.472173067393
Index: 106  Count: 16  Potential Energy:  -7739.5276

Index: 900  Count: 135  Potential Energy:  -7739.399250900709
Index: 907  Count: 136  Potential Energy:  -7739.4647537713445
Index: 913  Count: 137  Potential Energy:  -7739.49546889361
Index: 920  Count: 138  Potential Energy:  -7739.453147757404
Index: 927  Count: 139  Potential Energy:  -7739.365788829515
Index: 933  Count: 140  Potential Energy:  -7739.318708853982
Index: 940  Count: 141  Potential Energy:  -7739.349004795975
Index: 947  Count: 142  Potential Energy:  -7739.421361734061
Index: 953  Count: 143  Potential Energy:  -7739.44442057406
Index: 960  Count: 144  Potential Energy:  -7739.468361038256
Index: 967  Count: 145  Potential Energy:  -7739.519447328249
Index: 973  Count: 146  Potential Energy:  -7739.495192486611
Index: 980  Count: 147  Potential Energy:  -7739.404211646343
Index: 987  Count: 148  Potential Energy:  -7739.354141809947
Index: 993  Count: 149  Potential Energy:  -7739.377128506439
Index: 1000  Count: 150  Potential Energy:  -7739.4400145948175
Index: 

Index: 1774  Count: 266  Potential Energy:  -7739.53578161339
Index: 1780  Count: 267  Potential Energy:  -7739.478965058488
Index: 1787  Count: 268  Potential Energy:  -7739.430001499875
Index: 1794  Count: 269  Potential Energy:  -7739.429930926873
Index: 1800  Count: 270  Potential Energy:  -7739.478978674739
Index: 1807  Count: 271  Potential Energy:  -7739.561853104467
Index: 1814  Count: 272  Potential Energy:  -7739.617253815524
Index: 1820  Count: 273  Potential Energy:  -7739.640753219264
Index: 1827  Count: 274  Potential Energy:  -7739.616704794873
Index: 1834  Count: 275  Potential Energy:  -7739.514921971786
Index: 1840  Count: 276  Potential Energy:  -7739.391323532602
Index: 1847  Count: 277  Potential Energy:  -7739.2838664304545
Index: 1854  Count: 278  Potential Energy:  -7739.292171485745
Index: 1860  Count: 279  Potential Energy:  -7739.362257903494
Index: 1867  Count: 280  Potential Energy:  -7739.416462654108
Index: 1874  Count: 281  Potential Energy:  -7739.44802

Index: 2647  Count: 397  Potential Energy:  -7739.319750498192
Index: 2654  Count: 398  Potential Energy:  -7739.213077273464
Index: 2661  Count: 399  Potential Energy:  -7739.203227495846
Index: 2667  Count: 400  Potential Energy:  -7739.274534153262
Index: 2674  Count: 401  Potential Energy:  -7739.343541975508
Index: 2681  Count: 402  Potential Energy:  -7739.334251342728
Index: 2687  Count: 403  Potential Energy:  -7739.346345983419
Index: 2694  Count: 404  Potential Energy:  -7739.36929746563
Index: 2701  Count: 405  Potential Energy:  -7739.302283783549
Index: 2707  Count: 406  Potential Energy:  -7739.227160390924
Index: 2714  Count: 407  Potential Energy:  -7739.23067643033
Index: 2721  Count: 408  Potential Energy:  -7739.344987401203
Index: 2727  Count: 409  Potential Energy:  -7739.459791083553
Index: 2734  Count: 410  Potential Energy:  -7739.526158800146
Index: 2741  Count: 411  Potential Energy:  -7739.525438851962
Index: 2747  Count: 412  Potential Energy:  -7739.5342758

Index: 3514  Count: 527  Potential Energy:  -7739.280485706414
Index: 3521  Count: 528  Potential Energy:  -7739.285255479329
Index: 3528  Count: 529  Potential Energy:  -7739.3499759311035
Index: 3534  Count: 530  Potential Energy:  -7739.406661484407
Index: 3541  Count: 531  Potential Energy:  -7739.426232604079
Index: 3548  Count: 532  Potential Energy:  -7739.4293842355155
Index: 3554  Count: 533  Potential Energy:  -7739.437904091282
Index: 3561  Count: 534  Potential Energy:  -7739.404402293477
Index: 3568  Count: 535  Potential Energy:  -7739.338317290124
Index: 3574  Count: 536  Potential Energy:  -7739.329024102495
Index: 3581  Count: 537  Potential Energy:  -7739.40672342329
Index: 3588  Count: 538  Potential Energy:  -7739.500525425499
Index: 3594  Count: 539  Potential Energy:  -7739.5150208963305
Index: 3601  Count: 540  Potential Energy:  -7739.465024513724
Index: 3608  Count: 541  Potential Energy:  -7739.413451207869
Index: 3614  Count: 542  Potential Energy:  -7739.379

...

Index: 16366  Count: 2454  Potential Energy:  -7739.403405929803
Index: 16372  Count: 2455  Potential Energy:  -7739.520225647334
Index: 16379  Count: 2456  Potential Energy:  -7739.62353598035
Index: 16386  Count: 2457  Potential Energy:  -7739.696218887628
Index: 16392  Count: 2458  Potential Energy:  -7739.754764192796
Index: 16399  Count: 2459  Potential Energy:  -7739.726276419866
Index: 16406  Count: 2460  Potential Energy:  -7739.540867538364
Index: 16412  Count: 2461  Potential Energy:  -7739.380296714997
Index: 16419  Count: 2462  Potential Energy:  -7739.4101146549
Index: 16426  Count: 2463  Potential Energy:  -7739.609013080823
Index: 16432  Count: 2464  Potential Energy:  -7739.667251786461
Index: 16439  Count: 2465  Potential Energy:  -7739.595993656877
Index: 16446  Count: 2466  Potential Energy:  -7739.657238157194
Index: 16452  Count: 2467  Potential Energy:  -7739.646917501127
Index: 16459  Count: 2468  Potential Energy:  -7739.458764200376
Index: 16466  Count: 2469  P

Index: 17213  Count: 2581  Potential Energy:  -7739.5068553331375
Index: 17219  Count: 2582  Potential Energy:  -7739.5359339082515
Index: 17226  Count: 2583  Potential Energy:  -7739.561375983807
Index: 17233  Count: 2584  Potential Energy:  -7739.605694923187
Index: 17239  Count: 2585  Potential Energy:  -7739.661538510817
Index: 17246  Count: 2586  Potential Energy:  -7739.70938339365
Index: 17253  Count: 2587  Potential Energy:  -7739.658779069249
Index: 17259  Count: 2588  Potential Energy:  -7739.60188606336
Index: 17266  Count: 2589  Potential Energy:  -7739.525222910076
Index: 17273  Count: 2590  Potential Energy:  -7739.478134458331
Index: 17279  Count: 2591  Potential Energy:  -7739.529029073798
Index: 17286  Count: 2592  Potential Energy:  -7739.600869175388
Index: 17293  Count: 2593  Potential Energy:  -7739.548930398305
Index: 17299  Count: 2594  Potential Energy:  -7739.510104760961
Index: 17306  Count: 2595  Potential Energy:  -7739.543193563071
Index: 17313  Count: 2596

Index: 18060  Count: 2708  Potential Energy:  -7739.523190703494
Index: 18066  Count: 2709  Potential Energy:  -7739.400621343254
Index: 18073  Count: 2710  Potential Energy:  -7739.401937554759
Index: 18080  Count: 2711  Potential Energy:  -7739.502724314392
Index: 18086  Count: 2712  Potential Energy:  -7739.459661522171
Index: 18093  Count: 2713  Potential Energy:  -7739.351563082514
Index: 18100  Count: 2714  Potential Energy:  -7739.400745214572
Index: 18106  Count: 2715  Potential Energy:  -7739.527091967854
Index: 18113  Count: 2716  Potential Energy:  -7739.553964250537
Index: 18120  Count: 2717  Potential Energy:  -7739.495505931273
Index: 18126  Count: 2718  Potential Energy:  -7739.52351474887
Index: 18133  Count: 2719  Potential Energy:  -7739.555673542213
Index: 18140  Count: 2720  Potential Energy:  -7739.52174573827
Index: 18146  Count: 2721  Potential Energy:  -7739.518551615818
Index: 18153  Count: 2722  Potential Energy:  -7739.548295332584
Index: 18160  Count: 2723  

Index: 18907  Count: 2835  Potential Energy:  -7739.487737447726
Index: 18913  Count: 2836  Potential Energy:  -7739.494302835759
Index: 18920  Count: 2837  Potential Energy:  -7739.5764847591145
Index: 18927  Count: 2838  Potential Energy:  -7739.569593276089
Index: 18933  Count: 2839  Potential Energy:  -7739.500975459892
Index: 18940  Count: 2840  Potential Energy:  -7739.512137143546
Index: 18947  Count: 2841  Potential Energy:  -7739.501997432665
Index: 18953  Count: 2842  Potential Energy:  -7739.375069064525
Index: 18960  Count: 2843  Potential Energy:  -7739.267839234459
Index: 18967  Count: 2844  Potential Energy:  -7739.357104339013
Index: 18973  Count: 2845  Potential Energy:  -7739.462325772379
Index: 18980  Count: 2846  Potential Energy:  -7739.476060738468
Index: 18987  Count: 2847  Potential Energy:  -7739.509656657681
Index: 18993  Count: 2848  Potential Energy:  -7739.520421190613
Index: 19000  Count: 2849  Potential Energy:  -7739.425261328539
Index: 19007  Count: 285

Index: 19754  Count: 2962  Potential Energy:  -7739.587635266918
Index: 19760  Count: 2963  Potential Energy:  -7739.59075957094
Index: 19767  Count: 2964  Potential Energy:  -7739.522148053524
Index: 19774  Count: 2965  Potential Energy:  -7739.44974200909
Index: 19780  Count: 2966  Potential Energy:  -7739.4919724226465
Index: 19787  Count: 2967  Potential Energy:  -7739.593877816836
Index: 19794  Count: 2968  Potential Energy:  -7739.585707480464
Index: 19800  Count: 2969  Potential Energy:  -7739.569439553256
Index: 19807  Count: 2970  Potential Energy:  -7739.6053341765355
Index: 19814  Count: 2971  Potential Energy:  -7739.537659252523
Index: 19820  Count: 2972  Potential Energy:  -7739.435113602938
Index: 19827  Count: 2973  Potential Energy:  -7739.441491944311
Index: 19834  Count: 2974  Potential Energy:  -7739.557119106578
Index: 19840  Count: 2975  Potential Energy:  -7739.592340980556
Index: 19847  Count: 2976  Potential Energy:  -7739.593663425155
Index: 19854  Count: 2977

In [18]:
md_dft_rel_e_arr, dft_ref = get_relative_energy(md_dft_energy_arr)
print(dft_ref)
print(md_dft_rel_e_arr)

-7739.916532485042
[0.12667505 0.1641849  0.27096182 ... 0.4769672  0.48980128 0.45221071]


In [19]:
#md_dft_rel_energy_arr, relative_energy_ref_dft= get_relative_energy(md_dft_energy_arr)
md_dftb_rel_e_arr, relative_energy_ref_dftb = get_relative_energy(md_energy_arr)
#print(relative_energy_ref_dft)
print(relative_energy_ref_dftb)
print(md_dftb_rel_e_arr)
#print(md_dft_rel_energy_arr)

-390.0901277368928
[0.02086602 0.02918194 0.07036739 ... 0.33141117 0.35244398 0.29119134]


## Save the Bias ##

In [20]:
import pandas as pd
relative_energy = pd.Series([])
relative_energy['dft'] = dft_ref #relative_energy_ref_dft
relative_energy['calc'] = relative_energy_ref_dftb
#relative_energy.to_pickle('../relative_energy.pkl')

In [21]:
import WriteTrain
#test_name = 'rotating_120_180_append'

In [22]:
WriteTrain.write_train_samples(test_name, md_samplesArr, md_train_arr,
                               md_dftb_rel_e_arr, md_dft_rel_e_arr, 
                  relative_energy, 
                  md_energy_arr, md_dft_energy_arr, 
                  np.zeros( (1,1) ,dtype=np.float32), 
                    dft_force_arr, md_dft_force_arr)


md_samples_arr saved to  /home/junmian/Applications/YeltsinNet/md_capture/samples/glycine_190_400K_10000s/md_samples_arr.pkl
md_calc_rel_energy_arr saved to  /home/junmian/Applications/YeltsinNet/md_capture/samples/glycine_190_400K_10000s/md_calc_rel_e_arr_SCC.txt
rel_e_ref saved to /home/junmian/Applications/YeltsinNet/md_capture/samples/glycine_190_400K_10000s/rel_e_ref.pkl
md_high_rel_energy_arr saved to  /home/junmian/Applications/YeltsinNet/md_capture/samples/glycine_190_400K_10000s/md_high_rel_e_arr.txt


In [23]:
WriteTrain.write_train_samples

<function WriteTrain.write_train_samples(train_name, md_samples_arr, md_train_arr, md_calc_rel_e_arr, md_high_rel_e_arr, rel_e_ref, md_low_e_arr, md_high_e_arr, md_Mulliken, md_calc_force_arr, md_high_force_arr)>

In [24]:
#md_train_arr.to_pickle("/home/junmian/Applications/YeltsinNet/md_capture/samples/rotating_120_180_append/md_train_arr.pkl")

In [25]:
md_train_arr.to_pickle("/home/junmian/Applications/YeltsinNet/md_capture/samples/" + test_name + "/md_train_arr.pkl")